# Benchmark Results Visualization

This notebook displays the processed grid cells from the glacier monitoring benchmark runs.

In [5]:
from pathlib import Path
import geopandas as gpd
import folium
from folium import plugins

## Configure Output Directory

In [6]:
# Set your output directory here
OUTPUT_DIR = Path("../output/test_monitored_Right_10k")

# Find the grid file
grid_files = sorted(OUTPUT_DIR.glob("processed_grid_*.geojson"))

if grid_files:
    print(f"Found {len(grid_files)} grid file(s)")
    print(f"  Latest: {grid_files[-1].name}")
else:
    print("No grid files found!")

Found 1 grid file(s)
  Latest: processed_grid_20260202_110650.geojson


## Load and Display Grid

In [7]:
if grid_files:
    # Load the processed grid
    grid = gpd.read_file(grid_files[-1])
    print(f"Loaded grid with {len(grid)} cells")
    
    # Separate initial cells (with seeds) from expansion cells
    if 'seed_count' in grid.columns:
        initial_cells = grid[grid['seed_count'] > 0]
        expansion_cells = grid[grid['seed_count'] == 0]
        print(f"  Initial (with seeds): {len(initial_cells)}")
        print(f"  Expansion cells: {len(expansion_cells)}")
    else:
        initial_cells = grid
        expansion_cells = None
    
    # Calculate map center
    center_point = grid.geometry.union_all().centroid
    center_gdf = gpd.GeoDataFrame(geometry=[center_point], crs=grid.crs)
    center_wgs84 = center_gdf.to_crs(epsg=4326)
    center_lat = center_wgs84.geometry.iloc[0].y
    center_lon = center_wgs84.geometry.iloc[0].x
    
    # Convert to WGS84 for mapping
    initial_wgs84 = initial_cells.to_crs(epsg=4326) if len(initial_cells) > 0 else None
    expansion_wgs84 = expansion_cells.to_crs(epsg=4326) if expansion_cells is not None and len(expansion_cells) > 0 else None
    
    # Create map
    m = folium.Map(location=[center_lat, center_lon], zoom_start=8, tiles=None)
    
    # Add base layers
    folium.TileLayer('OpenStreetMap', name='OpenStreetMap').add_to(m)
    folium.TileLayer(
        tiles='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr='Esri',
        name='Satellite'
    ).add_to(m)
    
    # Tooltip fields
    tooltip_fields = ['cell_id']
    tooltip_aliases = ['Cell ID:']
    if 'snow_percentage' in grid.columns:
        tooltip_fields.append('snow_percentage')
        tooltip_aliases.append('Snow %:')
    
    # Add initial cells (with seeds) in BLUE
    if initial_wgs84 is not None and len(initial_wgs84) > 0:
        folium.GeoJson(
            initial_wgs84,
            name='Initial Cells (with seeds)',
            style_function=lambda x: {
                'fillColor': 'blue',
                'color': 'darkblue',
                'weight': 2,
                'fillOpacity': 0.4
            },
            tooltip=folium.GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases)
        ).add_to(m)
    
    # Add expansion cells in GREEN
    if expansion_wgs84 is not None and len(expansion_wgs84) > 0:
        folium.GeoJson(
            expansion_wgs84,
            name='Expansion Cells',
            style_function=lambda x: {
                'fillColor': 'lightgreen',
                'color': 'green',
                'weight': 2,
                'fillOpacity': 0.4
            },
            tooltip=folium.GeoJsonTooltip(fields=tooltip_fields, aliases=tooltip_aliases)
        ).add_to(m)
    
    plugins.Fullscreen().add_to(m)
    folium.LayerControl(collapsed=False).add_to(m)
    
    print(f"\n🔵 Blue: Initial cells (with seeds)")
    print(f"🟢 Green: Expansion cells")
    
    display(m)
else:
    print("No grid data to display.")

Loaded grid with 21 cells
  Initial (with seeds): 1
  Expansion cells: 20

🔵 Blue: Initial cells (with seeds)
🟢 Green: Expansion cells
